In [40]:
import torch
import pandas as pd
import numpy as np

In [91]:
path='./data/'
data = pd.read_csv(path+"ratings.dat", 
                   sep='::', 
                   names=["user_num", "item_num", "rating", "timestamp"], 
                   engine='python')
data.sort_values(by=["user_num", "timestamp"])
data

,user_num,item_num,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [92]:
view_per_user=data.groupby('user_num').item_num.count()
view_per_user=np.array(view_per_user)
print("min:",view_per_user.min())
print("max:",view_per_user.max())
print("mean:",view_per_user.mean())

min: 20
max: 2314
mean: 165.5975165562914


# 切割資料集 

In [110]:
test_data=data.groupby("user_num").tail(1)
train_data=data.drop(test_data.index)
test_data=test_data.reset_index(drop=True)

In [111]:
valid_data=train_data.groupby("user_num").tail(1)
train_data=train_data.drop(valid_data.index)
valid_data=valid_data.reset_index(drop=True)
train_data=train_data.reset_index(drop=True)
len(train_data)+len(valid_data)+len(test_data)==len(data)

True

# 是否跳號、對照表

In [57]:
print("user_num是否有跳號:",len(data["user_num"].unique())!=data["user_num"].max())
print("item_num是否有跳號:",len(data["item_num"].unique())!=data["item_num"].max())

user_num是否有跳號: False
item_num是否有跳號: True


In [58]:
#lookup table
user_lookup={v: i+1 for i, v in enumerate(data['user_num'].unique())}

sorted_movie_array=np.sort(data['item_num'].unique())
item_lookup={v: i+1 for i, v in enumerate(sorted_movie_array)}

# DataLoader 

In [118]:
from torch.utils.data import Dataset
class Interaction_Dataset(Dataset):
    def __init__(self, df, user_lookup, movie_lookup):
        self.df = df
        self.user_lookup = user_lookup
        self.movie_lookup = movie_lookup
        
        self.user_list=list()
        self.item_list=list()
        self.rating_list=list()
        
        for _ in range(len(self.df)):
            row = self.df.iloc[_]
            self.user_list.append(self.user_lookup[row.user_num])
            self.item_list.append(self.movie_lookup[row.item_num])
            self.rating_list.append(row.rating)
            
        self.user_tensor= torch.LongTensor(self.user_list)
        self.item_tensor= torch.LongTensor(self.item_list)
        self.rating_tensor = torch.FloatTensor(self.rating_list)

In [121]:
from torch.utils.data import DataLoader
class RateDataset(Dataset):
    def __init__(self, user_tensor, item_tensor, target_tensor):
        self.user_tensor = user_tensor
        self.item_tensor = item_tensor
        self.target_tensor = target_tensor

    def __getitem__(self, index):
        return self.user_tensor[index], self.item_tensor[index], self.target_tensor[index]
    
    def __len__(self):
        return self.user_tensor.size(0)

In [120]:
train_dataset=Interaction_Dataset(train_data,user_lookup,item_lookup)
valid_dataset=Interaction_Dataset(valid_data,user_lookup,item_lookup)
test_dataset=Interaction_Dataset(test_data,user_lookup,item_lookup)

In [122]:
train_data= RateDataset(train_dataset.user_tensor, train_dataset.item_tensor, train_dataset.rating_tensor)
valid_data= RateDataset(valid_dataset.user_tensor, valid_dataset.item_tensor, valid_dataset.rating_tensor)
test_data= RateDataset(test_dataset.user_tensor, test_dataset.item_tensor, test_dataset.rating_tensor)

# Model 

In [124]:
from torch import nn
class MfDot(nn.Module):

    def __init__(
        self, n_factors, n_users, n_items, ratings_range=None, 
    ):
        super().__init__()
        self.y_range = ratings_range
        self.user_embedding = nn.Embedding(n_users+1, n_factors, padding_idx=0)
        self.item_embedding = nn.Embedding(n_items+1, n_factors, padding_idx=0)
        
    def forward(self, inputs):
        users, items = inputs
        dot = self.user_embedding(users.to("cuda")) * self.item_embedding(items.to("cuda"))
        result = dot.sum(1)
        
        return(
                torch.sigmoid(result) * (self.y_range[1] - self.y_range[0])
                + self.y_range[0]
            )
    
    
        

In [150]:
def training_loop(n_epochs):
    global train_loss_list
    global valid_loss_list
    global test_loss_list
    
    #train_loop
    for epoch in range(n_epochs):
        batch_train_loss=0
        for bid, batch in enumerate(train_loader):
            model.train()
            u, i, r = batch[0], batch[1], batch[2].float()
            #forward
            pred=model([u,i])
            loss=loss_func(pred,r.to("cuda"))
            batch_train_loss+=loss
            # backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        val_loss=0
        for bid_, batch in enumerate(valid_loader):
            model.eval()
            u, i, r = batch[0], batch[1], batch[2].float()
            #forward
            pred=model([u,i])
            val_loss+=loss_func(pred,r.to("cuda"))
        
        test_loss=0
        for bid__, batch in enumerate(test_loader):
            model.eval()
            u, i, r = batch[0], batch[1], batch[2].float()
            #forward
            pred=model([u,i])
            test_loss+=loss_func(pred,r.to("cuda"))
        
            
        train_loss_list.append(loss.item()) 
        valid_loss_list.append(val_loss.item()/(bid_+1))
        test_loss_list.append(test_loss.item()/(bid__+1))
        
        print("-"*50)
        print('Epoch [{}/{}], train_Loss: {:.8f}, valid_Loss: {:.8f}, test_Loss: {:.8f}'
              .format(epoch + 1, n_epochs, batch_train_loss/(bid+1), val_loss.item()/(bid_+1), test_loss.item()/(bid__+1)))
        
        #early stopping
        if test_loss.item()/(bid__+1) > sum(test_loss_list[-5:])/len(test_loss_list[-5:]):
            print("Early Stop")
            break
    print("DONE")


In [151]:
batch_size=4096
val_batch_size=len(user_lookup)
emd_dim=512    #設定維度
epoch_num=150

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=val_batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=val_batch_size, shuffle=False)

#定義模型
model = MfDot(
        emd_dim, len(user_lookup), len(item_lookup), ratings_range=[1.0, 5.0]
    ).to("cuda")

for name, param in model.named_parameters():
        try:
            torch.nn.init.xavier_normal_(param.data)
        except:
            pass

#loss function
loss_func = torch.nn.MSELoss()
#optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

train_loss_list=[]
valid_loss_list=[]
test_loss_list=[]

training_loop(epoch_num)

--------------------------------------------------
Epoch [1/150], train_Loss: 1.58352804, valid_Loss: 1.76301146, test_Loss: 1.82083547
--------------------------------------------------
Epoch [2/150], train_Loss: 1.57663572, valid_Loss: 1.75784492, test_Loss: 1.81573582
--------------------------------------------------
Epoch [3/150], train_Loss: 1.55391943, valid_Loss: 1.72173178, test_Loss: 1.78132474
--------------------------------------------------
Epoch [4/150], train_Loss: 1.45777524, valid_Loss: 1.57054210, test_Loss: 1.63557553
--------------------------------------------------
Epoch [5/150], train_Loss: 1.23242497, valid_Loss: 1.30580819, test_Loss: 1.37293565
--------------------------------------------------
Epoch [6/150], train_Loss: 1.02437437, valid_Loss: 1.10954154, test_Loss: 1.17161274
--------------------------------------------------
Epoch [7/150], train_Loss: 0.92480242, valid_Loss: 1.00957477, test_Loss: 1.06622016
------------------------------------------------

# Predict 

In [337]:
for _ in list(user_lookup.keys()):
    u=torch.tensor([_])
    i=torch.tensor(list(item_lookup.values()))
    logit=-np.array(model([u,i]).tolist())

    rank_list=list(logit.argsort())
    item_list=list(item_lookup.keys())
    rated=list(data[data["user_num"]==_].item_num)
    
    top_N=10 #top_N
    read_index=0
    rec_list=[]
    prediction_rating=[]
    for i in range(top_N):
        item=item_list[rank_list[read_index]]
        while item in rated:
            read_index+=1
            item=item_list[rank_list[read_index]]

        rec_list.append(item)
        prediction_rating.append(logit[rank_list[read_index]])
        read_index+=1
    print(_,rec_list)
    prediction_rating = [round(-item, 2) for item in prediction_rating]
    print(_,prediction_rating)

1 [356, 318, 590, 593, 364, 1250, 1259, 1387, 920, 3101]
1 [4.56, 4.54, 4.49, 4.47, 4.46, 4.44, 4.41, 4.39, 4.39, 4.39]
2 [527, 2762, 260, 2324, 3114, 953, 1961, 1, 1097, 1172]
2 [4.74, 4.63, 4.63, 4.58, 4.5, 4.48, 4.48, 4.46, 4.46, 4.44]
3 [318, 110, 2028, 527, 858, 2329, 1262, 1201, 50, 1234]
3 [4.63, 4.6, 4.54, 4.49, 4.49, 4.48, 4.46, 4.45, 4.45, 4.45]
4 [1207, 858, 527, 318, 912, 2019, 2905, 923, 904, 1204]
4 [4.86, 4.85, 4.84, 4.84, 4.81, 4.8, 4.79, 4.78, 4.78, 4.78]
5 [922, 1148, 750, 1288, 1285, 1212, 745, 2019, 720, 926]
5 [4.53, 4.49, 4.48, 4.48, 4.37, 4.36, 4.36, 4.36, 4.36, 4.35]
6 [919, 2125, 899, 3147, 953, 1022, 3675, 1207, 905, 3061]
6 [4.42, 4.42, 4.41, 4.4, 4.4, 4.37, 4.36, 4.36, 4.36, 4.36]
7 [1198, 318, 527, 260, 2905, 1262, 50, 1036, 2019, 1214]
7 [4.83, 4.82, 4.77, 4.76, 4.75, 4.72, 4.71, 4.7, 4.68, 4.67]
8 [318, 260, 1198, 2905, 593, 50, 858, 1136, 1214, 1089]
8 [4.82, 4.78, 4.78, 4.76, 4.75, 4.74, 4.74, 4.71, 4.7, 4.69]
9 [1196, 260, 1198, 296, 858, 110, 356, 121

99 [858, 3552, 1221, 1387, 1196, 593, 1210, 1220, 1954, 1270]
99 [4.42, 4.39, 4.29, 4.27, 4.24, 4.16, 4.11, 4.08, 4.07, 4.06]
100 [899, 3022, 904, 1148, 910, 912, 3629, 3462, 922, 1212]
100 [4.31, 4.26, 4.21, 4.2, 4.18, 4.17, 4.15, 4.13, 4.12, 4.12]
101 [1101, 1270, 3753, 15, 168, 3147, 317, 1291, 1917, 315]
101 [4.87, 4.87, 4.86, 4.85, 4.85, 4.84, 4.84, 4.84, 4.84, 4.83]
102 [904, 2203, 908, 898, 903, 3022, 1254, 910, 1252, 2208]
102 [4.49, 4.41, 4.31, 4.27, 4.26, 4.26, 4.25, 4.24, 4.23, 4.21]
103 [260, 1196, 1198, 1210, 1240, 1036, 318, 1270, 2762, 3793]
103 [4.87, 4.8, 4.79, 4.7, 4.68, 4.67, 4.61, 4.59, 4.59, 4.58]
104 [2571, 1198, 260, 3578, 1196, 589, 2762, 1136, 110, 1036]
104 [4.74, 4.73, 4.72, 4.7, 4.68, 4.66, 4.61, 4.61, 4.6, 4.55]
105 [318, 1198, 904, 2905, 1207, 2019, 1148, 919, 527, 3338]
105 [4.9, 4.89, 4.86, 4.86, 4.86, 4.84, 4.83, 4.82, 4.81, 4.8]
106 [260, 1198, 1196, 527, 1, 318, 1207, 1148, 1035, 745]
106 [4.82, 4.8, 4.72, 4.71, 4.71, 4.66, 4.65, 4.64, 4.62, 4.61]
107

204 [1262, 1387, 2795, 1207, 2944, 3062, 904, 912, 2078, 2905]
204 [4.53, 4.48, 4.43, 4.42, 4.41, 4.37, 4.37, 4.36, 4.34, 4.32]
205 [745, 541, 1148, 2905, 3338, 2959, 1284, 720, 1963, 1223]
205 [4.6, 4.54, 4.51, 4.48, 4.44, 4.41, 4.41, 4.39, 4.38, 4.38]
206 [589, 1210, 1198, 3578, 3527, 110, 1374, 1240, 1036, 592]
206 [4.57, 4.44, 4.44, 4.32, 4.28, 4.22, 4.2, 4.17, 4.16, 4.12]
207 [318, 527, 50, 2905, 260, 593, 2028, 858, 2324, 2762]
207 [4.84, 4.8, 4.76, 4.74, 4.73, 4.72, 4.71, 4.7, 4.69, 4.68]
208 [858, 1213, 1221, 2905, 3683, 923, 1178, 2019, 1361, 922]
208 [4.65, 4.6, 4.59, 4.44, 4.4, 4.38, 4.35, 4.33, 4.29, 4.27]
209 [2571, 1136, 318, 50, 2028, 1198, 3578, 110, 1222, 1197]
209 [4.38, 4.24, 4.22, 4.19, 4.07, 4.05, 4.04, 4.04, 4.04, 4.01]
210 [2571, 2028, 356, 527, 318, 1198, 3578, 3147, 260, 110]
210 [4.91, 4.89, 4.86, 4.86, 4.85, 4.85, 4.85, 4.83, 4.82, 4.82]
211 [593, 1214, 1200, 2571, 1240, 2762, 780, 1198, 1219, 589]
211 [4.63, 4.56, 4.54, 4.53, 4.52, 4.52, 4.46, 4.45, 4.41, 4.

312 [1196, 527, 2905, 1210, 2324, 2028, 745, 1960, 858, 1357]
312 [4.3, 4.24, 4.16, 4.14, 4.13, 4.12, 4.12, 4.11, 4.09, 4.09]
313 [858, 2905, 2019, 750, 1207, 318, 1148, 745, 1178, 527]
313 [4.85, 4.83, 4.82, 4.81, 4.78, 4.78, 4.78, 4.77, 4.76, 4.76]
314 [858, 356, 1022, 1028, 943, 2081, 945, 3916, 2927, 911]
314 [4.34, 4.23, 4.21, 4.2, 4.19, 4.18, 4.17, 4.13, 4.13, 4.13]
315 [318, 50, 2762, 527, 1198, 1197, 2324, 2329, 2905, 110]
315 [4.89, 4.75, 4.74, 4.72, 4.68, 4.64, 4.64, 4.63, 4.63, 4.61]
316 [2628, 1356, 1198, 329, 2762, 527, 318, 3147, 1374, 3793]
316 [4.83, 4.81, 4.73, 4.73, 4.73, 4.67, 4.66, 4.66, 4.66, 4.65]
317 [2019, 750, 923, 904, 1204, 1178, 1212, 908, 1207, 541]
317 [4.65, 4.63, 4.58, 4.53, 4.5, 4.5, 4.47, 4.47, 4.44, 4.44]
318 [1148, 745, 2858, 260, 50, 2905, 1207, 720, 318, 1223]
318 [4.61, 4.61, 4.55, 4.54, 4.51, 4.48, 4.47, 4.44, 4.41, 4.4]
319 [2019, 904, 2905, 50, 3338, 3469, 3030, 954, 1267, 326]
319 [4.64, 4.59, 4.48, 4.47, 4.46, 4.43, 4.41, 4.39, 4.38, 4.38]
32

423 [810, 2555, 3593, 1739, 1323, 3574, 2643, 3268, 2807, 3945]
423 [4.68, 4.65, 4.61, 4.58, 4.52, 4.52, 4.49, 4.48, 4.47, 4.47]
424 [2019, 2905, 3022, 3338, 2203, 3077, 3091, 3801, 1148, 669]
424 [4.85, 4.82, 4.73, 4.72, 4.69, 4.68, 4.68, 4.67, 4.67, 4.66]
425 [1214, 1258, 1178, 1997, 608, 3019, 2351, 111, 1288, 928]
425 [4.16, 4.14, 4.1, 4.08, 4.07, 4.06, 4.05, 4.04, 4.02, 4.01]
426 [1136, 2905, 2019, 1148, 1252, 904, 3435, 1201, 1276, 1234]
426 [4.76, 4.75, 4.71, 4.64, 4.63, 4.61, 4.61, 4.61, 4.61, 4.6]
427 [260, 1196, 1198, 2571, 318, 1210, 50, 527, 2762, 1136]
427 [4.89, 4.84, 4.84, 4.78, 4.76, 4.74, 4.72, 4.72, 4.7, 4.7]
428 [296, 2959, 50, 2571, 1136, 2905, 1206, 745, 2502, 1148]
428 [4.81, 4.79, 4.76, 4.75, 4.73, 4.64, 4.63, 4.61, 4.6, 4.59]
429 [1230, 2905, 296, 1288, 1178, 922, 2019, 904, 3435, 910]
429 [4.72, 4.61, 4.61, 4.6, 4.57, 4.56, 4.55, 4.54, 4.54, 4.51]
430 [3114, 899, 1, 1035, 1197, 919, 2571, 1207, 1148, 1198]
430 [4.71, 4.68, 4.67, 4.64, 4.63, 4.59, 4.59, 4.57, 4.

...

5668 [3578, 780, 2571, 2762, 3147, 733, 597, 3753, 356, 1028]
5668 [4.67, 4.66, 4.64, 4.62, 4.61, 4.61, 4.56, 4.56, 4.56, 4.53]
5669 [1230, 923, 2947, 1252, 2949, 903, 3200, 912, 1304, 969]
5669 [4.02, 3.97, 3.94, 3.93, 3.9, 3.87, 3.85, 3.83, 3.83, 3.81]
5670 [110, 318, 296, 50, 589, 47, 1036, 2028, 1136, 1240]
5670 [4.94, 4.92, 4.89, 4.89, 4.88, 4.87, 4.85, 4.84, 4.84, 4.82]
5671 [2905, 858, 2762, 50, 2019, 3114, 1262, 1178, 1, 745]
5671 [4.86, 4.83, 4.82, 4.8, 4.79, 4.78, 4.76, 4.76, 4.76, 4.75]
5672 [50, 318, 1198, 2019, 260, 2905, 904, 745, 2571, 1148]
5672 [4.91, 4.88, 4.86, 4.85, 4.85, 4.85, 4.82, 4.82, 4.82, 4.82]
5673 [2019, 1207, 858, 1204, 904, 923, 1254, 930, 2905, 908]
5673 [4.58, 4.54, 4.51, 4.46, 4.44, 4.43, 4.42, 4.41, 4.41, 4.4]
5674 [1207, 1148, 750, 2019, 745, 923, 913, 1212, 904, 3429]
5674 [4.84, 4.82, 4.81, 4.8, 4.79, 4.78, 4.77, 4.75, 4.74, 4.74]
5675 [2019, 904, 1204, 2905, 1276, 1203, 750, 3469, 1253, 1927]
5675 [4.8, 4.79, 4.79, 4.75, 4.74, 4.71, 4.69, 4.66, 4.

5748 [1198, 260, 527, 50, 593, 2905, 356, 2324, 1197, 1262]
5748 [4.86, 4.84, 4.84, 4.78, 4.76, 4.75, 4.74, 4.74, 4.68, 4.67]
5749 [923, 750, 3134, 922, 1212, 904, 1260, 1213, 3089, 1228]
5749 [4.94, 4.91, 4.9, 4.88, 4.85, 4.84, 4.84, 4.83, 4.83, 4.83]
5750 [1207, 3134, 745, 3030, 904, 1204, 3022, 2920, 3095, 750]
5750 [4.43, 4.35, 4.33, 4.31, 4.31, 4.28, 4.28, 4.28, 4.26, 4.24]
5751 [1923, 3623, 1777, 2335, 3147, 1517, 1500, 593, 3565, 2145]
5751 [4.41, 4.21, 4.17, 4.16, 4.14, 4.13, 4.13, 4.13, 4.13, 4.12]
5752 [318, 2908, 1225, 2905, 668, 1288, 3338, 2064, 669, 1961]
5752 [4.47, 4.3, 4.3, 4.3, 4.26, 4.23, 4.22, 4.2, 4.19, 4.19]
5753 [1198, 2905, 1178, 750, 2019, 318, 541, 904, 1193, 50]
5753 [4.87, 4.87, 4.87, 4.86, 4.85, 4.84, 4.84, 4.83, 4.82, 4.81]
5754 [50, 2905, 750, 858, 745, 3897, 913, 3338, 1267, 306]
5754 [4.5, 4.47, 4.42, 4.39, 4.37, 4.36, 4.35, 4.33, 4.3, 4.29]
5755 [296, 608, 1213, 1178, 750, 1089, 3338, 1136, 1361, 50]
5755 [4.83, 4.8, 4.75, 4.73, 4.68, 4.64, 4.62, 4.58,

5826 [1206, 1213, 1178, 541, 3000, 2905, 668, 3338, 1223, 1394]
5826 [4.68, 4.47, 4.46, 4.46, 4.45, 4.42, 4.39, 4.38, 4.38, 4.35]
5827 [1196, 260, 1198, 2905, 318, 50, 2571, 110, 2762, 1210]
5827 [4.7, 4.67, 4.67, 4.58, 4.54, 4.53, 4.53, 4.45, 4.41, 4.41]
5828 [2019, 1148, 858, 745, 1198, 50, 527, 750, 2905, 1196]
5828 [4.81, 4.77, 4.76, 4.76, 4.76, 4.76, 4.75, 4.74, 4.74, 4.73]
5829 [50, 318, 2019, 2905, 750, 1207, 1148, 296, 1193, 745]
5829 [4.7, 4.57, 4.54, 4.51, 4.48, 4.47, 4.46, 4.45, 4.45, 4.43]
5830 [318, 2324, 1, 3147, 2762, 527, 356, 2501, 3916, 3753]
5830 [4.83, 4.76, 4.75, 4.75, 4.74, 4.7, 4.69, 4.69, 4.64, 4.62]
5831 [2905, 1207, 3338, 3469, 1131, 3668, 678, 1945, 2064, 3222]
5831 [4.71, 4.68, 4.55, 4.55, 4.54, 4.47, 4.45, 4.44, 4.44, 4.43]
5832 [260, 1148, 2905, 1198, 50, 745, 2762, 720, 1223, 1193]
5832 [4.7, 4.59, 4.58, 4.57, 4.52, 4.51, 4.47, 4.45, 4.42, 4.42]
5833 [1148, 745, 1035, 1172, 1207, 1307, 2762, 2905, 1246, 213]
5833 [4.72, 4.72, 4.72, 4.71, 4.69, 4.68, 4.66,

5897 [2905, 318, 527, 260, 858, 2019, 1148, 1198, 1193, 1207]
5897 [4.9, 4.9, 4.89, 4.89, 4.88, 4.88, 4.86, 4.86, 4.85, 4.85]
5898 [608, 1148, 3114, 318, 745, 2905, 1193, 720, 2959, 296]
5898 [4.34, 4.32, 4.26, 4.26, 4.22, 4.22, 4.15, 4.11, 4.1, 4.09]
5899 [260, 1198, 318, 858, 1196, 110, 1197, 50, 1210, 953]
5899 [4.47, 4.44, 4.43, 4.42, 4.34, 4.33, 4.32, 4.31, 4.29, 4.29]
5900 [608, 1199, 2076, 2997, 2427, 306, 1178, 1206, 750, 2064]
5900 [4.59, 4.49, 4.49, 4.48, 4.46, 4.46, 4.44, 4.42, 4.41, 4.41]
5901 [260, 1035, 3148, 2150, 110, 3489, 1353, 282, 720, 3844]
5901 [4.31, 4.18, 4.0, 3.97, 3.96, 3.95, 3.94, 3.91, 3.91, 3.91]
5902 [858, 923, 608, 1228, 111, 904, 3134, 2019, 922, 3634]
5902 [4.62, 4.61, 4.57, 4.46, 4.46, 4.45, 4.45, 4.44, 4.43, 4.43]
5903 [1198, 2019, 904, 2905, 913, 745, 1224, 953, 1178, 2716]
5903 [4.89, 4.82, 4.76, 4.72, 4.7, 4.7, 4.68, 4.68, 4.67, 4.66]
5904 [318, 750, 1206, 2905, 858, 745, 2019, 527, 1148, 2858]
5904 [4.42, 4.38, 4.35, 4.35, 4.35, 4.34, 4.33, 4.32, 

5985 [1198, 260, 1148, 2905, 745, 904, 1196, 2019, 720, 750]
5985 [4.73, 4.72, 4.69, 4.66, 4.65, 4.64, 4.63, 4.62, 4.61, 4.6]
5986 [527, 745, 1148, 2905, 318, 50, 58, 1299, 2997, 720]
5986 [4.89, 4.87, 4.84, 4.84, 4.84, 4.83, 4.81, 4.79, 4.79, 4.78]
5987 [750, 1212, 923, 922, 3134, 1178, 1252, 2019, 3338, 3435]
5987 [4.77, 4.75, 4.7, 4.68, 4.66, 4.66, 4.65, 4.65, 4.6, 4.59]
5988 [1288, 858, 162, 750, 1136, 1222, 527, 3751, 668, 923]
5988 [4.06, 4.05, 3.95, 3.95, 3.92, 3.91, 3.9, 3.89, 3.87, 3.87]
5989 [318, 3147, 527, 2571, 593, 260, 2905, 1198, 50, 2028]
5989 [4.78, 4.75, 4.73, 4.71, 4.71, 4.68, 4.67, 4.66, 4.65, 4.62]
5990 [923, 3134, 912, 2019, 910, 3022, 904, 3307, 1178, 908]
5990 [4.79, 4.67, 4.67, 4.65, 4.63, 4.62, 4.6, 4.59, 4.58, 4.58]
5991 [905, 1207, 1148, 912, 1949, 1296, 933, 3022, 745, 2019]
5991 [4.36, 4.36, 4.36, 4.33, 4.31, 4.29, 4.28, 4.27, 4.27, 4.25]
5992 [318, 50, 296, 527, 110, 858, 2324, 1262, 3578, 1193]
5992 [4.86, 4.81, 4.75, 4.71, 4.71, 4.69, 4.67, 4.62, 4.62,

# And More...

In [381]:
cos = nn.CosineSimilarity(dim=0, eps=1e-6)
for u in range(len(user_lookup)):
    user=model.user_embedding(torch.tensor(u).to("cuda"))

    sim_list=[]
    for i in range(len(user_lookup)):
        if i ==u :continue
        b=model.user_embedding(torch.tensor(i).to("cuda"))
        sim_list.append(cos(user, b).item())
    print(max(sim_list))

0.1714310199022293
0.5012321472167969
0.4205075204372406
0.4473259150981903
0.6976060271263123
0.41704633831977844
0.48711127042770386
0.6200962662696838
0.540941596031189
0.41919058561325073
0.5689260959625244
0.4257744252681732
0.4340139627456665
0.370035320520401
0.3825046420097351
0.33791279792785645
0.4319928288459778
0.5826178789138794
0.5007246136665344
0.3588453531265259
0.6375062465667725
0.34308168292045593
0.3613187372684479
0.29152539372444153
0.5438622236251831
0.3542461395263672
0.3204945921897888
0.5751553773880005
0.40564101934432983
0.3646168112754822
0.31516993045806885
0.365711510181427
0.3876695930957794
0.32391878962516785
0.48608705401420593
0.41456231474876404
0.5760208964347839
0.4805787205696106
0.4273490607738495
0.5418184995651245
0.4831782281398773
0.45394331216812134
0.4359734058380127
0.5858061909675598
0.4677264094352722
0.26573890447616577
0.44012728333473206
0.4590170383453369
0.36409980058670044
0.4624451994895935
0.38983213901519775
0.4735189974308014

0.6016483902931213
0.49065449833869934
0.4105645418167114
0.47390082478523254
0.5766952037811279
0.33162644505500793
0.45494315028190613
0.4768076539039612
0.4548477232456207
0.49123960733413696
0.5892586708068848
0.44154030084609985
0.5090957880020142
0.46658557653427124
0.5045145750045776
0.4388578236103058
0.4809267520904541
0.47521260380744934
0.4815767705440521
0.7986288666725159
0.7698476910591125
0.521827757358551
0.5250301957130432
0.47842898964881897
0.6891557574272156
0.7001247406005859
0.4433598816394806
0.5484145283699036
0.5770426988601685
0.3734044134616852
0.5170601606369019
0.4262290298938751
0.4493979513645172
0.29978516697883606
0.39254841208457947
0.39766332507133484
0.5372189283370972
0.3930914103984833
0.5243289470672607
0.5236309766769409
0.3338667154312134
0.45178788900375366
0.5708396434783936
0.47518226504325867
0.5026670098304749
0.5448748469352722
0.5201382040977478
0.45458102226257324
0.5275388956069946
0.4538136124610901
0.40366026759147644
0.48961699008941

0.35107284784317017
0.40586137771606445
0.4293300211429596
0.529051661491394
0.5515801906585693
0.430878221988678
0.2835162580013275
0.5775653719902039
0.3467032015323639
0.5693034529685974
0.4540480375289917
0.4617024064064026
0.47499173879623413
0.3642905652523041
0.4449133276939392
0.3652249574661255
0.48094794154167175
0.6198800802230835
0.3808790147304535
0.4012397825717926
0.5355377197265625
0.6422561407089233
0.47680822014808655
0.41801443696022034
0.7356935143470764
0.5096831917762756
0.5701826214790344
0.32814157009124756
0.7356935143470764
0.4144613742828369
0.6155841946601868
0.5879849791526794
0.44390663504600525
0.46257859468460083
0.4349209666252136
0.3785182535648346
0.506991446018219
0.5111883282661438
0.5067041516304016
0.41795364022254944
0.703761637210846
0.5698027610778809
0.5377363562583923
0.4028577506542206
0.6132142543792725
0.4701012969017029
0.48456770181655884
0.45269858837127686
0.7204030752182007
0.4099227488040924
0.28669434785842896
0.5442027449607849
0.6

0.5565953254699707
0.4165801703929901
0.4151627719402313
0.48227837681770325
0.5245108604431152
0.5451168417930603
0.39004355669021606
0.5740007162094116
0.6148766279220581
0.4203258454799652
0.5569140315055847
0.35593080520629883
0.46820303797721863
0.4185246229171753
0.43647193908691406
0.43910497426986694
0.5074629187583923
0.4057711064815521
0.5563898682594299
0.41362830996513367
0.6290772557258606
0.4221086800098419
0.5043014883995056
0.639293909072876
0.611305832862854
0.6595918536186218
0.43813201785087585
0.4193631112575531
0.45145896077156067
0.3792894184589386
0.46484118700027466
0.40100497007369995
0.44084686040878296
0.6345280408859253
0.393075168132782
0.44363924860954285
0.41517215967178345
0.3586479425430298
0.6361002326011658
0.33806484937667847
0.49913522601127625
0.48577675223350525
0.5052700638771057
0.4654361605644226
0.38549068570137024
0.44671830534935
0.4401560425758362
0.42334386706352234
0.5474521517753601
0.4518982470035553
0.3765373229980469
0.534802079200744

0.35289302468299866
0.5327048301696777
0.4364054799079895
0.38363802433013916
0.37819206714630127
0.4675542414188385
0.48102372884750366
0.44118770956993103
0.46403342485427856
0.7067768573760986
0.2946096360683441
0.5532720685005188
0.376085102558136
0.5984684824943542
0.4628615975379944
0.5440847277641296
0.5221571922302246
0.5560219883918762
0.6108419895172119
0.5140784978866577
0.3913112282752991
0.5128870010375977
0.33751392364501953
0.6860134601593018
0.39394381642341614
0.5651115775108337
0.42690303921699524
0.3241148293018341
0.4463878273963928
0.5883828997612
0.4717818796634674
0.48459526896476746
0.5507694482803345
0.49107199907302856
0.6051209568977356
0.5094624757766724
0.481282114982605
0.6314483284950256
0.54169100522995
0.3630216121673584
0.4212214946746826
0.29429399967193604
0.6182992458343506
0.440827339887619
0.4343462288379669
0.3479071259498596
0.42753270268440247
0.4723670184612274
0.6310083270072937
0.5226081013679504
0.43215301632881165
0.46010735630989075
0.285

...

0.44584396481513977
0.46089503169059753
0.5130739808082581
0.550909698009491
0.4086490273475647
0.3674830496311188
0.3822838068008423
0.4943466782569885
0.5171762108802795
0.33496758341789246
0.40004968643188477
0.3696771562099457
0.49985331296920776
0.5293504595756531
0.5176281332969666
0.5414740443229675
0.45844122767448425
0.4338019788265228
0.6731641888618469
0.34687796235084534
0.45576462149620056
0.5726267695426941
0.6376189589500427
0.5433369874954224
0.4946245849132538
0.3397384583950043
0.5903183817863464
0.49205002188682556
0.4230717420578003
0.5765048861503601
0.4367889165878296
0.40620648860931396
0.5098788738250732
0.43616819381713867
0.4797237515449524
0.43701171875
0.27871742844581604
0.39311322569847107
0.6080214381217957
0.5302709937095642
0.5266745090484619
0.3579418659210205
0.5127701759338379
0.5336512923240662
0.5367320775985718
0.6204776167869568
0.6301494240760803
0.40209513902664185
0.398770272731781
0.6334313154220581
0.3676765263080597
0.3967932164669037
0.659

0.40411195158958435
0.523985743522644
0.4771938920021057
0.4663579761981964
0.26031649112701416
0.6269644498825073
0.4007624685764313
0.49174416065216064
0.42316603660583496
0.2773398756980896
0.5279601812362671
0.42776212096214294
0.43119072914123535
0.5832036733627319
0.4623844623565674
0.5269593000411987
0.6260318160057068
0.6107569932937622
0.5726786851882935
0.4904283881187439
0.6423998475074768
0.5515938997268677
0.7384811043739319
0.43229934573173523
0.497601717710495
0.3517141342163086
0.5859040021896362
0.5199226140975952
0.5141844153404236
0.4537792205810547
0.6929144859313965
0.5429355502128601
0.47161751985549927
0.40818607807159424
0.6052076816558838
0.32257914543151855
0.40300095081329346
0.5002230405807495
0.6457936763763428
0.4187213182449341
0.47577619552612305
0.5245998501777649
0.4425401985645294
0.37802067399024963
0.4195011258125305
0.586469829082489
0.5498772263526917
0.49987781047821045
0.5014909505844116
0.5917270183563232
0.41053321957588196
0.4771665632724762


0.4021473526954651
0.6437290906906128
0.5784671306610107
0.503268301486969
0.40999189019203186
0.43738195300102234
0.6388842463493347
0.3629436492919922
0.3821430206298828
0.37805917859077454
0.513414740562439
0.3763268291950226
0.5497279763221741
0.4393007457256317
0.3788122236728668
0.4135007858276367
0.461557000875473
0.6116912364959717
0.4940965175628662
0.48428210616111755
0.41970011591911316
0.5073815584182739
0.328542560338974
0.6037552952766418
0.33079296350479126
0.6044524908065796
0.5683140158653259
0.47603514790534973
0.5345888733863831
0.555083155632019
0.5366930961608887
0.51502925157547
0.3843502104282379
0.545717179775238
0.6469907760620117
0.6744832396507263
0.5937180519104004
0.6651825904846191
0.3984359800815582
0.5734235048294067
0.5194559097290039
0.36843085289001465
0.36184439063072205
0.5933468341827393
0.5526142716407776
0.35287776589393616
0.3641984164714813
0.49330464005470276
0.5023151636123657
0.521955132484436
0.5628243088722229
0.4990515112876892
0.55746853

0.5756889581680298
0.5922301411628723
0.3806535303592682
0.36606431007385254
0.442162424325943
0.4648422300815582
0.55592280626297
0.5654111504554749
0.3277701735496521
0.6008723974227905
0.38919058442115784
0.5266936421394348
0.29418089985847473
0.4365498423576355
0.376166433095932
0.4943782389163971
0.3765307366847992
0.5049692392349243
0.46337103843688965
0.5246449112892151
0.5844730138778687
0.4934203028678894
0.6208279132843018
0.6382019519805908
0.532599687576294
0.40405845642089844
0.5226794481277466
0.5341091156005859
0.5310418009757996
0.4529573619365692
0.694212019443512
0.3594299256801605
0.6659538745880127
0.5633218884468079
0.6402720808982849
0.5062344670295715
0.44667863845825195
0.49080556631088257
0.298457533121109
0.5221002697944641
0.45545220375061035
0.5270369052886963
0.3170429766178131
0.5486604571342468
0.47555825114250183
0.3777291774749756
0.3028786778450012
0.4849800169467926
0.3714144229888916
0.4535621404647827
0.6776329278945923
0.5869686603546143
0.65330231

0.5150114297866821
0.364096999168396
0.5359307527542114
0.43316686153411865
0.3426845967769623
0.45340561866760254
0.39956140518188477
0.5642980933189392
0.5086257457733154
0.47713300585746765
0.6857001185417175
0.3768728971481323
0.5795893669128418
0.35842829942703247
0.45189404487609863
0.5865333080291748
0.6373118162155151
0.39301520586013794
0.5004875063896179
0.2629149854183197
0.2760757505893707
0.4072781205177307
0.31028369069099426
0.5038152933120728
0.5409305691719055
0.3911377787590027
0.5862757563591003
0.5443025827407837
0.4107743799686432
0.3797328770160675
0.388607919216156
0.52386075258255
0.42714765667915344
0.6508947014808655
0.4073615074157715
0.4949670135974884
0.45912131667137146
0.5911790728569031
0.5163102149963379
0.4207356870174408
0.5259796380996704
0.351485937833786
0.6412346363067627
0.5766078233718872
0.42520907521247864
0.5806028842926025
0.4299640357494354
0.6095677614212036
0.42931559681892395
0.5069868564605713
0.37891915440559387
0.5043709874153137
0.43